# Coleta de Dados para Análise do WDO

Este notebook demonstra o processo de coleta de dados para análise do mini dólar (WDO).

In [63]:
# Importações necessárias
import os
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta

# Configura caminhos
PROJECT_ROOT = Path(os.getcwd()).parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Importa módulos do projeto
from src.data.collect_market_data import fetch_wdo_data
from src.data.economic_indicators import EconomicDataCollector

# Configuração direta da chave API
FRED_API_KEY = "e531b375faba60b9228efde0901f08cc"

# Configura pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Configuração concluída!")

Configuração concluída!


## 1. Definir período de análise

In [64]:
# Define período de análise
end_date = datetime.now()
start_date = end_date - timedelta(days=30)

# Cria instância do coletor
collector = EconomicDataCollector()

print(f"Período de análise: {start_date.date()} até {end_date.date()}")

Período de análise: 2024-11-02 até 2024-12-02


## 2. Coletar dados do BCB

In [71]:
# Teste direto da API do BCB
import requests
from datetime import datetime, timedelta
import json

# Define o período
end_date = datetime.now()
start_date = end_date - timedelta(days=30)
ipca_start_date = end_date - timedelta(days=180)  # 6 meses para IPCA

# Dicionário com as configurações de cada série
series = {
    'SELIC': {
        'codigo': 432,
        'start_date': start_date,
        'descricao': 'Taxa SELIC'
    },
    'IPCA': {
        'codigo': 433,
        'start_date': ipca_start_date,
        'descricao': 'Índice de Preços ao Consumidor Amplo'
    },
    'Câmbio': {
        'codigo': 1,
        'start_date': start_date,
        'descricao': 'Taxa de Câmbio - Livre - Dólar americano (compra)'
    }
}

collected_data = {}  # Para armazenar os DataFrames

for nome, config in series.items():
    print(f"\n=== Coletando {nome} ({config['descricao']}) ===")
    
    # Formata as datas no padrão do BCB
    start = config['start_date'].strftime('%d/%m/%Y')
    end = end_date.strftime('%d/%m/%Y')
    
    # Monta a URL com formato JSON explícito
    url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{config["codigo"]}/dados'
    params = {
        'formato': 'json',
        'dataInicial': start,
        'dataFinal': end
    }
    
    headers = {
        'Accept': 'application/json'
    }
    
    print(f"Período: {start} até {end}")
    
    try:
        response = requests.get(url, params=params, headers=headers)
        print(f"Status Code: {response.status_code}")
        
        if response.ok:
            try:
                if 'application/json' in response.headers.get('Content-Type', ''):
                    data = response.json()
                else:
                    # Tenta forçar interpretação como JSON
                    data = response.json()
                
                if data:  # Verifica se há dados
                    df = pd.DataFrame(data)
                    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
                    df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
                    df = df.set_index('data')
                    print("\nDados processados:")
                    print(df.head())
                    
                    collected_data[nome] = df  # Armazena o DataFrame
                else:
                    print("Nenhum dado retornado para o período")
                    
            except json.JSONDecodeError as e:
                # Se falhar, tenta uma requisição alternativa
                alternative_url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{config["codigo"]}/dados/ultimos/30'
                alt_response = requests.get(alternative_url, params={'formato': 'json'})
                
                if alt_response.ok:
                    data = alt_response.json()
                    df = pd.DataFrame(data)
                    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
                    df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
                    df = df.set_index('data')
                    print("\nDados processados (via URL alternativa):")
                    print(df.head())
                    
                    collected_data[nome] = df  # Armazena o DataFrame
                else:
                    print(f"Erro também na URL alternativa: {alt_response.status_code}")
                    
            except Exception as e:
                print(f"Erro ao processar dados: {str(e)}")
        else:
            print(f"Erro na requisição: {response.text[:200]}")
            
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição HTTP: {str(e)}")

# Mostra resumo dos dados coletados
print("\n=== Resumo dos Dados Coletados ===")
for nome, df in collected_data.items():
    print(f"\n{nome}:")
    print(f"Período: {df.index.min()} até {df.index.max()}")
    print(f"Número de registros: {len(df)}")


=== Coletando SELIC (Taxa SELIC) ===
Período: 02/11/2024 até 02/12/2024
Status Code: 200

Dados processados:
            valor
data             
2024-11-02  10.75
2024-11-03  10.75
2024-11-04  10.75
2024-11-05  10.75
2024-11-06  10.75

=== Coletando IPCA (Índice de Preços ao Consumidor Amplo) ===
Período: 05/06/2024 até 02/12/2024
Status Code: 200

Dados processados:
            valor
data             
2024-06-01   0.21
2024-07-01   0.38
2024-08-01  -0.02
2024-09-01   0.44
2024-10-01   0.56

=== Coletando Câmbio (Taxa de Câmbio - Livre - Dólar americano (compra)) ===
Período: 02/11/2024 até 02/12/2024
Status Code: 200

Dados processados:
             valor
data              
2024-11-04  5.7898
2024-11-05  5.7846
2024-11-06  5.7648
2024-11-07  5.6624
2024-11-08  5.7648

=== Resumo dos Dados Coletados ===

SELIC:
Período: 2024-11-02 00:00:00 até 2024-12-02 00:00:00
Número de registros: 31

IPCA:
Período: 2024-06-01 00:00:00 até 2024-10-01 00:00:00
Número de registros: 5

Câmbio:
Período

## 3. Coletar dados do FRED

In [72]:
# Teste da API do FRED
print("=== Teste FRED ===")
url = "https://api.stlouisfed.org/fred/series/observations"
params = {
    'series_id': 'DFF',  # Federal Funds Rate
    'api_key': FRED_API_KEY,
    'file_type': 'json',
    'observation_start': start_date.strftime('%Y-%m-%d'),
    'observation_end': end_date.strftime('%Y-%m-%d')
}

print("\nTestando Federal Funds Rate:")
print(f"URL: {url}")
print(f"Parâmetros: {params}")

response = requests.get(url, params=params)
print(f"\nStatus Code: {response.status_code}")

if response.ok:
    data = response.json()
    if 'observations' in data:
        df = pd.DataFrame(data['observations'])
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date')
        print("\nDados coletados:")
        print(df.head())
    else:
        print("\nResposta não contém observações:")
        print(data)
else:
    print("\nErro na resposta:")
    print(response.text)

=== Teste FRED ===

Testando Federal Funds Rate:
URL: https://api.stlouisfed.org/fred/series/observations
Parâmetros: {'series_id': 'DFF', 'api_key': 'e531b375faba60b9228efde0901f08cc', 'file_type': 'json', 'observation_start': '2024-11-02', 'observation_end': '2024-12-02'}

Status Code: 200

Dados coletados:
           realtime_start realtime_end value
date                                        
2024-11-02     2024-12-01   2024-12-01  4.83
2024-11-03     2024-12-01   2024-12-01  4.83
2024-11-04     2024-12-01   2024-12-01  4.83
2024-11-05     2024-12-01   2024-12-01  4.83
2024-11-06     2024-12-01   2024-12-01  4.83


## 4. Salvar dados coletados

In [73]:
# Cria diretório se não existir
os.makedirs('../data/raw', exist_ok=True)

# Salva dados do BCB
for name, df in bcb_data.items():
    df.to_csv(f'../data/raw/{name}_data.csv')
    print(f'Dados salvos: {name}_data.csv')

# Salva dados do FRED se disponíveis
if fred_api_key:
    for name, df in fred_data.items():
        df.to_csv(f'../data/raw/{name}_data.csv')
        print(f'Dados salvos: {name}_data.csv')